In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15925,2024-12-19,Brasil Nbb,19:30,Pato,São José,1.72,1.95,160.5,1.84,1.82,-2.5,1.95,1.72,fJmqetGU,0.581395,0.512821,0.543478,0.549451,0.094216,0.8,0.4,NaN,NaN,308.542,156.798840,0.508193,132.620,13.444651,0.101377,186.424,237.578,404.79,128.25,1.0,0.0,0.0,0.0,0.088629,0.007728,0.088629,-0.46,-0.092,-7.826087,0.573833,0.4,-0.173833,3.46,0.692,1.372832,0.398954,0.4,0.001046
15926,2024-12-19,Brasil Nbb,19:30,Unifacisa,Fortaleza B.C.,1.17,4.20,150.5,1.88,1.78,-13.5,1.95,1.72,Y3azc2oI,0.854701,0.238095,0.531915,0.561798,0.092796,0.6,0.8,NaN,NaN,160.584,72.975288,0.454437,226.750,139.057969,0.613266,134.238,293.062,77.88,219.78,0.0,0.0,0.0,0.0,0.797964,0.038640,0.088629,-3.48,-0.696,-0.244253,0.711701,0.4,-0.311701,-5.00,-1.000,-3.200000,0.382381,0.2,-0.182381
15927,2024-12-19,Eua Nba,21:00,Detroit Pistons,Utah Jazz,1.54,2.58,227.5,1.87,1.97,-4.5,1.98,1.88,jTkrIH1E,0.649351,0.387597,0.534759,0.507614,0.036948,0.6,1.0,NaN,NaN,540.100,253.500657,0.469359,489.352,208.404194,0.425878,240.418,479.540,333.75,392.69,1.0,0.0,0.0,0.0,0.356986,0.036828,0.036638,1.46,0.292,1.849315,0.417762,0.5,0.082238,-2.78,-0.556,-2.841727,0.270731,0.3,0.029269
15928,2024-12-19,Eua Nba,21:00,Orlando Magic,Oklahoma City Thunder,3.08,1.41,208.5,1.86,1.98,6.5,1.96,1.90,bggjGeWQ,0.324675,0.709220,0.537634,0.505051,0.033895,0.4,0.4,NaN,NaN,213.024,52.004157,0.244123,166.800,30.314837,0.181744,190.990,172.526,235.69,159.46,0.0,0.0,0.0,0.0,0.525999,0.044194,0.021983,1.70,0.340,6.117647,0.662070,0.9,0.237930,0.75,0.150,2.733333,0.694447,0.7,0.005553
15929,2024-12-19,Eua Nba,21:00,Washington Wizards,Charlotte Hornets,2.46,1.59,225.5,1.82,2.03,3.5,1.95,1.91,GAZMAZgl,0.406504,0.628931,0.549451,0.492611,0.035435,0.6,0.6,NaN,NaN,842.680,408.747398,0.485056,495.180,218.794868,0.441849,636.664,473.928,718.34,286.90,1.0,0.0,0.0,0.0,0.303794,0.077139,0.014655,2.50,0.500,2.920000,0.231724,0.2,-0.031724,0.89,0.178,3.314607,0.306474,0.2,-0.106474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15989,2024-12-19,Uruguai Liga Uruguaia,20:15,Urunday,Nacional,3.07,1.30,164.5,1.83,1.83,6.5,1.97,1.71,GMkcg6Ph,0.325733,0.769231,0.546448,0.546448,0.094964,0.0,0.6,NaN,NaN,173.908,52.757107,0.303362,122.854,33.051909,0.269034,0.000,170.748,212.22,177.30,0.0,0.0,0.0,0.0,0.572805,0.000000,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,2.05,0.410,0.731707,0.577436,0.8,0.222564
15990,2024-12-19,Uruguai Liga Uruguaia,21:15,Penarol,Defensor Sporting,1.56,2.21,159.5,1.83,1.83,-4.5,1.99,1.70,KfdleSft,0.641026,0.452489,0.546448,0.546448,0.093514,0.6,0.6,NaN,NaN,111.668,33.981765,0.304311,131.610,23.957018,0.182030,112.022,158.726,91.96,161.33,0.0,0.0,0.0,0.0,0.243830,0.000000,0.111144,0.75,0.150,3.733333,0.738072,0.6,-0.138072,-1.84,-0.368,-3.288043,0.551526,0.5,-0.051526
15991,2024-12-19,Eua Ncaa,21:00,North Alabama,Charleston Southern,1.23,3.97,146.5,1.83,1.83,-9.5,1.83,1.83,M19YFMe4,0.813008,0.251889,0.546448,0.546448,0.064897,0.4,0.8,NaN,NaN,133.530,67.308713,0.504072,498.554,724.340706,1.452883,120.792,746.624,99.22,209.22,0.0,0.0,0.0,0.0,0.745182,0.000000,0.000000,1.30,0.260,0.884615,0.651135,0.7,0.048865,1.40,0.280,10.607143,0.171870,0.1,-0.071870
15992,2024-12-19,Eua Ncaa,21:30,Middle Tenn. St.,Lipscomb,1.77,2.10,147.5,1.80,1.95,-3.5,2.05,1.71,Ey6QHrRi,0.56

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Detroit Pistons,Utah Jazz,227.5,1.87,0.9736,Over
1,21:00,Eua Nba,Orlando Magic,Oklahoma City Thunder,208.5,1.86,0.9811,Over
2,21:00,Eua Nba,Washington Wizards,Charlotte Hornets,225.5,1.82,0.9840,Over
3,21:30,Eua Nba,Boston Celtics,Chicago Bulls,242.5,1.82,0.9194,Over
4,21:30,Eua Nba,Toronto Raptors,Brooklyn Nets,218.5,1.85,0.9251,Over
5,22:00,Eua Nba,Houston Rockets,New Orleans Pelicans,217.5,1.87,0.8929,Over
6,22:00,Eua Nba,Memphis Grizzlies,Golden State Warriors,234.5,1.84,0.8557,Over
7,22:00,Eua Nba,San Antonio Spurs,Atlanta Hawks,232.5,1.87,0.8717,Over
8,22:30,Eua Nba,Dallas Mavericks,Los Angeles Clippers,225.5,1.85,0.8993,Over
9,23:00,Eua Nba,Phoenix Suns,Indiana Pacers,234.5,1.83,0.9269,Over
